<a href="https://colab.research.google.com/github/insublee/Ke-T5-for-Question-Generation-/blob/main/T5_QA_QG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. install and import

In [1]:
!pip install --quiet pytorch-lightning wandb hydra-core optuna datasets transformers sentencepiece rouge_score sacrebleu deepspeed
!pip install --upgrade nltk

#1. data

In [2]:
from typing import Optional, Dict, List
import os
from omegaconf import DictConfig
import datasets
from datasets import load_dataset, concatenate_datasets, load_metric
import logging
import torch
import pytorch_lightning as pl
from pytorch_lightning.plugins import DeepSpeedPlugin
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything
from torch.utils.data import DataLoader
from torch.nn.modules.loss import CrossEntropyLoss
from transformers import (
    AdamW,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

config = DictConfig({
    "name":"model_v0.0",#돌린 버젼. 돌릴때마다 바꿔주세요
    "model_name":"KETI-AIR/ke-t5-small-ko",
    # "model_name":"KETI-AIR/ke-t5-base-ko",
    "entity":"insublee",
    "train_batch_size":30,
    "eval_batch_size":40,
    "data_usage_percent":10,#dev용으로 10%만 사용해서 테스트해보고 나중엔 100%사용
    "tasks":["qa","qg","ans_ext"],#테스크를 따라 데이터셋 생성.
    "remove_columns":["answers", "context", "id", "question", "title"],#데이터컨버팅 후 지울 컬럼
    "loader_columns":['attention_mask', 'input_ids', 'labels'],#모델에 입력하는 컬럼. 텐서로변환해줄때 사용
    "max_source_length":512,#해당길이 기준으로 토큰길이가 길면자르고 짧으면 패딩
    "max_target_length":32,
    "hl_token":"<hl>",
    "sep_token":"<sep>",
    "num_proc":os.cpu_count(),#dataset 처리와 데이터로더에서 사용할 워커 갯수
    "qa_prefix":"answer the question. question: ",#정답 생성시 소스데이터 앞에 붙이는 프리픽스
    "qg_prefix":"generate question. answer: ",#질문 생성시 소스데이터 앞에 붙이는 프리픽스
    "ans_ext_prefix":"extract answer. context: ",#정답 후보 추출시 소스데이터 앞에 붙이는 프리픽스
    "metrics":["bleu", "sacrebleu", "meteor", "rouge"],#단어중첩기반 평가지표
    "learning_rate":2e-5,#모델이 학습할 러닝레이트.
    "adam_epsilon":1e-8,#옵티마이저 AdamW에서 사용하는 파라미터.
    "weight_decay":1e-2,#모델의파라미터중에 "bias", "LayerNorm.weight" 빼고 디케이 적용 (과적합 방지용) 
    "scheduler_warmup_steps":500,#스케쥴러에 적용할 웜업스텝
    "total_steps":30000,#총30000 스탭 학습. 에폭기준보다 이쪽이 직관적임. 
    "scheduler_interval_step":"step",#러닝레이트를 관리하는 스케쥴러의 작동 인터벌
    "scheduler_frequency":1,
    "gpus":1,#코랩이니까 1
    "monitor":"val_loss",#val_loss기준으로 저장 및 얼리스타핑
    "project":"T5_multi_task",#wandb 프로젝트 이름
    "save_path":"./save/",#저장경로
    "save_top_k":2,#성능 제일 좋았던 두개 저장
    "monitor_mode":"min",#monitor 기준으로 작은것 저장
    "num_beams":5,
    "repetition_penalty":2.5,
    "length_penalty":1.0,
    "deepspeed":{
        "fp16": {
            "enabled": True,
            "loss_scale": 0,
            "loss_scale_window": 1000,
            "hysteresis": 2,
            "min_loss_scale": 1
        },
        "zero_optimization": {
            "stage": 2,
            "allgather_partitions": True,
            "allgather_bucket_size": 2e8,
            "reduce_scatter": True,
            "reduce_bucket_size": 2e8,
            "overlap_comm": True,
            "contiguous_gradients": True,
            "cpu_offload": True,
        },
    }
})


class KorQuadV1DataModule(pl.LightningDataModule):
    def __init__(
        self, 
        config:DictConfig,
        ):
        super().__init__()
        self.config = config
        self.logger = logging.getLogger("DataModuleLogger")


    def prepare_data(self):
        """data load"""

        self.dataset = load_dataset(
            "squad_kor_v1",
            split={
                'train':f'train[:{self.config.data_usage_percent}%]', 
                'validation':f'validation[:{self.config.data_usage_percent}%]'
                }
            )
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)


    def setup(
        self, 
        stage: Optional[str] = None
        ) -> None:
        """
        data tokenizing for each tasks
        TODO : -100 for labels
        """
        
        for split in self.dataset.keys():
            # create dataset for eatch tasks and concatenate alltogether
            # in case tasks = ['qa', 'qg', 'ext_ans'], dataset going to be X3
            self.dataset[split] = concatenate_datasets(
                [
                 self.dataset[split].map(
                     self._convert_to_task(task),
                     batched=True,
                     num_proc=self.config.num_proc,
                     remove_columns=self.config.remove_columns
                     )
                 for task in self.config.tasks
                 ]
                )
            
            self.dataset[split] = self.dataset[split].map(
                self._convert_to_features,
                batched=True,
                remove_columns=[c for c in self.dataset[split].column_names if c not in self.config.loader_columns],
                num_proc=self.config.num_proc
            )
            self.dataset[split].set_format(type="torch", columns=self.config.loader_columns)

    def _convert_to_task(
        self,
        task:str,
        ):
        if task=="qa":
            return self._qa_formatter
        elif task=="qg":
            return self._qg_formatter
        elif task=="ans_ext":
            return self._ans_ext_formatter
        else:
            raise Exception("plz select one of qa, qg, ans_ext")

    def _qa_formatter(
        self,
        example_batch:datasets.arrow_dataset.Batch, 
        )->Dict:
        """
        QA
        question: What is the answer to life? context: 42 is the answer tolife, the universe and everything.
        42
        """
        return_dict = {}
        return_dict["source_text"] = [self.config.qa_prefix + q + " context: " + c + ' </s>'
                                      for q,c 
                                      in zip(example_batch['question'], example_batch['context'])]
        if example_batch.get('answers') is not None:
            if isinstance(example_batch['answers'][0], dict):
                return_dict["target_text"] = [i['text'][0] for i in example_batch['answers']]
            else:
                return_dict["target_text"] = example_batch['answers']
        return return_dict

    def _qg_formatter(
        self,
        example_batch:datasets.arrow_dataset.Batch, 
        )->Dict:
        """
        Answer aware QG
        generate question. answer : 42 context: 42 is the answer to life, the universe and everything.
        What is the answer to life?
        """
        return_dict = {}
        if isinstance(example_batch['answers'][0], dict):
            answers = [i['text'][0] for i in example_batch['answers']]
        else:
            answers = example_batch['answers']
        return_dict["source_text"] = [self.config.qg_prefix + a + " context: " + c + ' </s>'
                                      for a,c 
                                      in zip(answers, example_batch['context'])]
        if example_batch.get('question') is not None:
            return_dict["target_text"] = example_batch['question']
        return return_dict

    def _ans_ext_formatter(
        self,
        example_batch:datasets.arrow_dataset.Batch, 
        )->Dict:
        """
        Answer extraction
        extract answer. context: 42 is the answer to life, the universe and everything.
        42
        """
        return_dict = {}
        return_dict["source_text"] = [self.config.ans_ext_prefix + c + ' </s>'
                                      for c 
                                      in example_batch['context']]
        if example_batch.get('answers') is not None:
            if isinstance(example_batch['answers'][0], dict):
                return_dict["target_text"] = [i['text'][0] for i in example_batch['answers']]
            else:
                return_dict["target_text"] = example_batch['answers']
        return return_dict


    def _convert_to_features(
        self, 
        example_batch:List, 
        indices=None
        )->Dict:

        source_encoding = self.tokenizer.batch_encode_plus(
            example_batch["source_text"],
            max_length=self.config.max_source_length,
            padding="max_length",
            pad_to_max_length=True,
            truncation=True,
        )
        target_encoding = self.tokenizer.batch_encode_plus(
            example_batch["target_text"],
            max_length=self.config.max_target_length,
            padding="max_length",
            pad_to_max_length=True,
            truncation=True,
        )

        encodings = {
            "input_ids": source_encoding["input_ids"],
            "labels": target_encoding["input_ids"],
            "attention_mask": source_encoding["attention_mask"],
        }

        return encodings

    
    def train_dataloader(self):
        return self._dataloader('train')


    def val_dataloader(self):
        return self._dataloader('validation')


    def test_dataloader(self):
        return self._dataloader('test')


    def _dataloader(
        self, 
        split:str,
        ):
        batch_size = self.config.train_batch_size if split=='train' else self.config.eval_batch_size
        shuffle = True if split=='train' else False

        return DataLoader(
            self.dataset[split], 
            batch_size = batch_size,
            shuffle = shuffle,
            pin_memory=True,
            num_workers=self.config.num_proc,
            )


dm = KorQuadV1DataModule(config)
dm.prepare_data()
dm.setup("fit")
# train_batch = next(iter(dm.train_dataloader()))
# for k in train_batch.keys():
#     print(k, train_batch[k].size())
# [dm.tokenizer.decode(i) for i in train_batch['input_ids']]

Reusing dataset squad_kor_v1 (/root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725/cache-c1c4c817253f127c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725/cache-5b16e4798029157f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725/cache-ec6625ce0bc3169e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725/cache-f16ca1ad52663996.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725/cache-bfd67da93a198374.arrow
Loading cached proce

In [3]:
train_batch = next(iter(dm.train_dataloader()))
for k in train_batch.keys():
    print(k, train_batch[k].size())
[dm.tokenizer.decode(i) for i in train_batch['input_ids']]

attention_mask torch.Size([20, 512])
input_ids torch.Size([20, 512])
labels torch.Size([20, 32])


['answer the question. question: 강희제의 아버지는? context: 1661년(순치 18년) 부친인 순치제가 천연두로 붕어하자, 여덟 살의 어린 나이에 황제로 즉위하여 1722년(강희 61년)까지 61년간 재위함으로써 중국 역사상 가장 긴 재위기간을 가진 황제이다. 즉위 초에는 어린 나이로 인해 병부상서 오배 등의 네 명의 보정대신들에게 국정을 일임하였다. 만주족의 예법에 따라 1667년(강희 6년), 14세에 친정을 시작하였으며 2년 뒤에 발생한 오배의 난을 평정하여 실질적으로 조정을 장악하였다. 이후에는 1673년(강희 12년)에 8년 동안 지속된 삼번의 난을 평정하고 이어 1683년(강희 22년)에 대만을 청나라에 복속시켰으며 북방의 러시아와 네르친스크 조약을 체결함으로써 만주와 연해주 쪽의 국경을 확정하였다. 한편 1696년(강희 35년)에는 청나라에 대항하던 몽골 오이라트로 친히 원정하여 중가르의 칸 가르단의 군사를 격파하여 북방을 안정되게 하였으며, 1721년(강희 60년)에는 가르단을 지원했던 티베트에 군대를 보내 정복함으로써 티베트를 청나라 영토에 편입시켰다. 이렇듯 강희제는 오배의 난과 삼번의 난과 같은 재위 전반기의 위기를 극복하고 대외 정책에서의 외교적, 정치적 성공을 거둠으로써 강력한 절대 황제권을 수립하였다. </s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

#2.model

In [4]:
class MultiTaskTransformer(LightningModule):
    def __init__(
        self, 
        config:DictConfig,
        ):
        super().__init__()
        self.config = config
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)
        self.metrics = {metric:load_metric(metric) for metric in self.config.metrics}

    
    def forward(self, **inputs):
        return self.model(**inputs)


    def generate(self, batch):
        generated = self.model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            max_length=self.config.max_target_length,
            min_length=self.config.max_target_length, # forced to generate
            num_beams=self.config.num_beams,
            repetition_penalty=self.config.repetition_penalty,
            length_penalty=self.config.length_penalty,
            # early_stopping=True,
            use_cache=True,
            )
        return generated


    def Interaction(
        self, 
        dm:pl.LightningDataModule, 
        format:str, 
        batch:Dict[str, List[str]]):

        assert batch.get("context")!=None, f"context is needed. but get {batch.keys()}"

        if format=="qg" and batch.get("answers")!=None:
            # 정답과 질문을 qg 포맷으로 변경
            batch = dm._qg_formatter(batch)

        elif format=="qg" and batch.get("answers")==None:
            # qg를 선택했지만 정답을 안준경우 (batch->ans_ext->qg)
            answers = self.Interaction(dm, "ans_ext", batch)
            batch["answers"]=answers
            return self.Interaction(dm, "qg", batch)
        
        elif format=="qa":
            assert batch.get("question")!=None, f"you choose {format}, context and question is needed. but get {batch.keys()}"
            # 컨택스트와 질문을 qa 포맷으로 변경
            batch = dm._qa_formatter(batch)

        elif format=="ans_ext":
            # ans_ext 포맷으로 변경
            batch = dm._ans_ext_formatter(batch)
        
        else:
            raise Exception("plz select one of qa, qg, ans_ext")

        source_encoding = self.tokenizer.batch_encode_plus(
            batch["source_text"],
            max_length=self.config.max_source_length,
            padding="max_length",
            pad_to_max_length=True,
            truncation=True,
            return_tensors='pt',
        )
        source_encoding = source_encoding.to(self.device)

        generated_tensors = self.generate(source_encoding)

        result = self.tokenizer.batch_decode(
            generated_tensors,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
            )
        return result

    
    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs[0]
        return loss


    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss = outputs[0]
        generated_tensors = self.generate(batch)
        assert generated_tensors.size(1)==self.config.max_target_length, f"get odd tensor from generation. generated_tensors.size : {generated_tensors.size()}"
        assert batch['labels'].size(1)==self.config.max_target_length, f"get odd tensor from labels. generated.size : {batch['labels'].size()}"

        return {"val_loss": val_loss, "predictions": generated_tensors, "references": batch['labels']}


    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        predictions = torch.cat([x["predictions"] for x in outputs]).detach().cpu()
        references = torch.cat([x["references"] for x in outputs]).detach().cpu()

        predictions = self.tokenizer.batch_decode(
            predictions,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
            )
        references = self.tokenizer.batch_decode(
            references,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
            )
        
        metric_results = self.compute_score(predictions, references)
        self.log("val_loss", val_loss, prog_bar=True)
        for met in "bleu sacrebleu meteor rouge1 rouge2 rougeL".split():
            self.log(f"{met}", metric_results[met], prog_bar=True)


    def compute_score(
        self,
        predictions:List[str],
        references:List[str],
        )->Dict:
        
        results={}
        for metric in self.config.metrics:
            if metric=='bleu':
                predictions = [i.split() for i in predictions]
                references = [[i.split()] for i in references]
                # predictions = [["hello", "there", "general", "kenobi"],["foo", "bar", "foobar"]
                # references = [[["hello", "there", "general", "kenobi", "?!"]], [["foo", "bar", "foobar"]]
                results['bleu'] = self.metrics[metric].compute(predictions=predictions, references=references)['bleu']
            
            elif metric=='sacrebleu':
                references = [[i] for i in references]
                # predictions = ["hello there general kenobi", "foo bar foobar"] # 생성
                # references = [["hello there general kenobi"], ["hello there !"]] #정답
                results['sacrebleu'] = self.metrics[metric].compute(predictions=predictions, references=references)['score']
            
            elif metric=='meteor':
                # predictions = ["It is a guide to action which ensures that the military always obeys the commands of the party"]
                # references = ["It is a guide to action that ensures that the military will forever heed Party commands"]
                results['meteor'] = self.metrics[metric].compute(predictions=predictions, references=references)['meteor']
            
            elif metric=='rouge':
                # predictions = ["hello there", "general kenobi"]
                # references = ["hello there", "general ?!"]
                rouge = self.metrics[metric].compute(predictions=predictions, references=references)
                results['rouge1'] = rouge['rouge1'].mid.fmeasure
                results['rouge2'] = rouge['rouge2'].mid.fmeasure
                results['rougeL'] = rouge['rougeL'].mid.fmeasure
                
        return results


    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.config.learning_rate, eps=self.config.adam_epsilon)

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.config.scheduler_warmup_steps,
            num_training_steps=self.config.total_steps,
        )
        scheduler = {"scheduler": scheduler, "interval": self.config.scheduler_interval_step, "frequency": self.config.scheduler_frequency}
        return [optimizer], [scheduler]


model = MultiTaskTransformer(config)
# res = model(**train_batch)
# training_result = model.training_step(train_batch,0)
# validation_result = model.validation_step(train_batch,0)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#3. tunning

In [7]:
logger = WandbLogger(
    name=config.name,
    project=config.project
)

callbacks = [
    LearningRateMonitor(
        logging_interval="step",
        log_momentum=False,
        ),
    ModelCheckpoint(
        monitor=config.monitor,
        dirpath=config.save_path,
        filename="model.epoch={epoch:02d}.val_loss={val_loss:.3f}",
        save_top_k=config.save_top_k,
        mode=config.monitor_mode,
    ),
    EarlyStopping(config.monitor)
]

trainer = pl.Trainer(
    gpus=config.gpus,
    max_steps=config.total_steps,
    logger=logger,
    callbacks=callbacks,
    #https://github.com/huggingface/transformers/issues/8771#issuecomment-759176685
    plugins=DeepSpeedPlugin(config.deepspeed),
)

trainer.fit(model, datamodule=dm)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:325: LightningDeprecationWarning: Passing <pytorch_lightning.plugins.training_type.deepspeed.DeepSpeedPlugin object at 0x7f919096c850> `strategy` to the `plugins` flag in Trainer has been deprecated in v1.5 and will be removed in v1.7. Use `Trainer(strategy=<pytorch_lightning.plugins.training_type.deepspeed.DeepSpeedPlugin object at 0x7f919096c850>)` instead.
  f"Passing {plug} `strategy` to the `plugins` flag in Trainer has been deprecated"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
initializing deepspeed distributed: GLOBAL_RANK: 0, MEMBER: 1/1
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/plugins/training_type/deepspeed.py:626: UserWarning: Inferring the batch size for internal deepspeed logging from the `train_dataloader()`. If you require skipping this, please pass `Trainer(strategy=DeepSpeedPlugin(logging

[2021-11-14 05:56:08,339] [WARNING] [engine.py:903:_configure_optimizer] **** You are using ZeRO with an untested optimizer, proceed with caution *****
Using /root/.cache/torch_extensions/py37_cu111 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py37_cu111/utils/build.ninja...
Building extension module utils...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module utils...
Time to load utils op: 0.38155245780944824 seconds
Rank: 0 partition count [1, 1] and sizes[(76895232, False), (384, False)] 
Using /root/.cache/torch_extensions/py37_cu111 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...
Time to load utils op: 0.0010952949523925781 seconds


wandb: Currently logged in as: insublee (use `wandb login --relogin` to force relogin)



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 76.9 M
-----------------------------------------------------
76.9 M    Trainable params
0         Non-trainable params
76.9 M    Total params
307.582   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


#4. visualize

In [8]:
%wandb {config.entity}/{config.project}/ -h 1024

#5. test sample

In [5]:
data = load_dataset("squad_kor_v1",split={'validation':f'validation[:{1}%]'})
sample = data['validation'][0]
print(sample)
context = sample['context']
answers = sample['answers']
question = sample['question']

Reusing dataset squad_kor_v1 (/root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/1 [00:00<?, ?it/s]

{'id': '6548850-0-0', 'title': '임종석', 'context': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.', 'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?', 'answers': {'text': ['1989년 2월 15일'], 'answer_start': [0]}}


In [6]:
qg_with_answer_sample = {"context":[context],"answers":[answers]}
qg_with_answer_generated = model.Interaction(dm,"qg", qg_with_answer_sample)
print("qg_with_answer_generated : ", qg_with_answer_generated,'\n')

qg_with_no_answer_sample = {"context":[context]}
qg_with_no_answer_generated = model.Interaction(dm,"qg", qg_with_no_answer_sample)
print("qg_with_no_answer_generated : ", qg_with_answer_generated,'\n')

qa_sample = {"context":[context],"question":[question]}
qa_generated = model.Interaction(dm,"qa", qa_sample)
print("qa_generated : ", qg_with_answer_generated,'\n')

ans_ext_sample = {"context":[context]}
ans_ext_generated = model.Interaction(dm, "ans_ext", ans_ext_sample)
print("ans_ext_generated : ", qg_with_answer_generated,'\n')

qg_with_answer_generated :  ['동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을'] 

qg_with_no_answer_generated :  ['동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을'] 

qa_generated :  ['동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을'] 

ans_ext_generated :  ['동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을 동점을'] 

